In [1]:
spark.stop()

In [2]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import from_json, col, count, desc, window



In [ ]:
# Define schema
schema = StructType([
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("transaction_id", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("category", StringType(), True),
    StructField("product", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("payment_type", StringType(), True)
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("TopTrendingProducts") \
    .master("local[*]") \
    .config("spark.jars", "/usr/local/spark/jars/spark-sql-kafka-0-10_2.11-2.4.8.jar,/usr/local/spark/jars/kafka-clients-2.7.0.jar") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Read Kafka stream
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "transactions-topic") \
    .option("startingOffsets", "latest") \
    .load()

# Extract and parse JSON
json_df = kafka_df.selectExpr("CAST(value AS STRING) as json_str")
parsed_df = json_df.select(from_json(col("json_str"), schema).alias("data")).select("data.*")

# Aggregate: Top 5 trending products by count
top_products_df = parsed_df.groupBy("product") \
    .agg(count("*").alias("product_count")) \
    .orderBy(desc("product_count")) \
    .limit(5)

# Write to console every 5 seconds
query = top_products_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='10 seconds') \
    .option("truncate", False) \
    .start()

query = top_products_df.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("top_products") \
    .trigger(processingTime='10 seconds') \
    .start()

query.awaitTermination()